# Geração de arquivos
Na hora de entregar o projeto as 3 primeiras células desse notebook em um arquivo .py para ter o script gerador dos arquivos de base do programa

### Importando as bibliotecas

In [1]:
#importa as bibliotecas
import pandas as pd
import xlrd
import pickle #modulo usado para serializar dados para o arquivo binario
import os
import time
from unicodedata import normalize #biblioteca usada para criar as arvores TRIE

#Le o excel para a variavel df
df = pd.read_excel('Hot Stuff.xlsx')

### Definição das funções que serão usadas para gerar os arquivos

In [2]:
#Funcao que retira as features de artistas selecionados e salva apenas o artista principal da musica
def retiraFeaturing(artistaString):
    
    if " Featuring" in artistaString:   # Se a música tiver um featuring, deve-se retirá-lo
            indexFeat = artistaString.find(" Featuring")   # pega índice do char space antes de Featuring
            artistaString = artistaString[:indexFeat]   # retorna apenas o primeiro artista
        
    return artistaString   # caso não caia no if, simplesmente retorna o artista assim como veio

# Função que formata string desconsiderando maiúsculas e minúsculas e que remove acentos
def formataString(string):
    string = normalize('NFKD', string).encode('ASCII', 'ignore').decode('ASCII')
    return string.upper()

#------------------------------------------------------------
#Definicao das funcoes usadas para gerar as arvores TRIE
#------------------------------------------------------------
#Definicao da classe de nodo das arvores TRIE
class TrieNode(object):
    def __init__(self, char: str):   # __init__ é um método especial para fazer construtores
        self.char = char   # caractere do nodo atual
        self.filhos = []   # nodos filhos
        self.pFinalizada = False   # se é o último nodo e a palavra terminou
        self.indices = []   # lista vazia para nodos que não são término de palavra
        ### não é um índice único e sim uma lista, pois podem existir títulos de música repetidos com mais de um artista

#Definicao da funcao que adiciona uma string na arvore TRIE
def addString(raiz, palavra: str, indice: int):   # função usada para adicionar uma palavra nova à estrutura trie 
    palavra = formataString(palavra)   # formata string de entrada para o padrão (todas as letras maiúsculas sem acento)
    nodo = raiz
    for char in palavra:
        encontradoEmFilho = False
        # busca pelo caractere nos filhos do nodo atual
        for filho in nodo.filhos:
            if filho.char == char: 
                nodo = filho   # apontamos o nodo para o filho que contém esse char
                encontradoEmFilho = True
                break
        
        if not encontradoEmFilho:   # se o caractere não foi encontrado, adiciona novo filho
            novoNodo = TrieNode(char)
            nodo.filhos.append(novoNodo)
            nodo = novoNodo   # apontamos, então, o nodo para seu novo filho e continuamos a iteração
    
    nodo.pFinalizada = True   # indica que até ali pode ser uma palavra (nome de música/artista completo)
    nodo.indices.append(indice)   # assinala o indice passado
    
#Funcao que procura uma string na arvore TRIE 
def findString(select, dados, raiz, palavra: str) -> (bool, []):
    """
      1. Se a palavra existe e tem algum índice associado, retorna verdadeiro e a lista de índices dela
      2. Se a palavra não existe, retorna falso e uma lista com sugestões de palavras
      
      Sendo o parâmetro select utilizada para indicar se a busca é por artista ou música (0 música, 1 artista)
    """
    palavra = formataString(palavra)   # formata string de entrada para o padrão (todas as letras maiúsculas sem acento)
    nodo = raiz
    stringSugerida = ""  # guarda os caracteres já encontrados para sugerir os próximos
    
    if not raiz.filhos:   # se o nodo raiz não tiver nenhum filho, trivial, retorna falso
        return False, []
    for char in palavra:
        charNaoEncontrado = True
        for filho in nodo.filhos:
            if filho.char == char:
                charNaoEncontrado = False   # assinala que o char foi encontrado
                stringSugerida = stringSugerida+char
                nodo = filho   # passa iteração para o nodo filho
                break
        
        if charNaoEncontrado:   # chama a função que procura sugestões e retorna a tupla (False, lista de sugestões)
            return False, sugereStrings(select, dados, nodo, stringSugerida, 15)
    
    # Caso passe por todos os caracteres sem retornar false, então significa que a palavra foi encontrada
    # Resta saber se aquele nodo é um nodo final com um índice associado
    if nodo.pFinalizada:
        return True, nodo.indices
    else:
        return False, sugereStrings(select, dados, nodo, stringSugerida, 15)
    
#Funcoes utilizadas para sugerir buscas
def sugereStrings(select, dados, nodo, string, maxSugestoes):   # onde select igual a 1 significa que se procura um artista e False, música
        listaDeSugestoes = []   # cria lista vazia para adicionarmos as sugestões
        procuraSugestoes(nodo, string, listaDeSugestoes)   # chama a função que percorre a trie procurando sugestões
        
        
        itera = copy.deepcopy(listaDeSugestoes)   # utilizado para iteração na lista que está sendo modificada enquanto o loop itera
        
        # substitui os indices pelos títulos
        for indice in itera:
            listaDeSugestoes.remove(indice)   # remove o índice 
            if select:
                listaDeSugestoes.append(dados[indice]['Artista']) 
            else:
                listaDeSugestoes.append(dados[indice]['Titulo']) 
            
        
        listaDeSugestoes = sorted(listaDeSugestoes)   # ordena a lista alfabeticamente
        return listaDeSugestoes[:maxSugestoes]   # retorna apenas os n elementos da lista definidos por maxSugestoes

    
# função que dado um nodo, uma parte correta de uma string (que existe na TRIE), uma lista de sugestoes, retorna uma lista de sugestões
def procuraSugestoes(nodo, string, listaDeSugestoes):
        
        if nodo.pFinalizada:   
            listaDeSugestoes.append(nodo.indices[0])   # adiciona nova sugestão na lista
        
        for filho in nodo.filhos:
            
            procuraSugestoes(filho, string+filho.char, listaDeSugestoes)   # recursão para cada filho 
            if filho.pFinalizada:   
                listaDeSugestoes.append(filho.indices[0])   # adiciona nova sugestão na lista
                
        return  
  
# Função auxiliar para adicionar artista levando-se em consideração a retirada do possível "feat" da string
# A função checa se a string passada como parâmetro contém uma sinalização de featuring para evitar que se crie um "artista novo"
'''
def adicionaArtista(raiz, artista, indice):
    
    if " Featuring" in artista:   # Se a música tiver um featuring, deve-se retirá-lo
        indexFeat = artista.find(" Featuring")
        counter = 0
        artistaNewString = ""
        
        # como artista = [:indexFeat] dá erro de sintaxe, precisa-se iterar e construir outra string 
        # um loop for com contador (e break) é necessário nesse caso para poder iterar pela sting 
        for char in artista:
            if counter == indexFeat:
                break
            artistaNewString = artistaNewString + char
            counter += 1

        artista = artistaNewString   # atualiza nome do artista     

    addString(raiz, artista, indice)
    '''

'\ndef adicionaArtista(raiz, artista, indice):\n    \n    if " Featuring" in artista:   # Se a música tiver um featuring, deve-se retirá-lo\n        indexFeat = artista.find(" Featuring")\n        counter = 0\n        artistaNewString = ""\n        \n        # como artista = [:indexFeat] dá erro de sintaxe, precisa-se iterar e construir outra string \n        # um loop for com contador (e break) é necessário nesse caso para poder iterar pela sting \n        for char in artista:\n            if counter == indexFeat:\n                break\n            artistaNewString = artistaNewString + char\n            counter += 1\n\n        artista = artistaNewString   # atualiza nome do artista     \n\n    addString(raiz, artista, indice)\n    '

### Código que gera os arquivos

In [13]:
#esta celula calcula o quao popular foi uma musica no mes especificado (Maio de 1990) atribuindo um sistema de pontos baseado em que posicao ela ficou no top 100 em cada semana do mês
#vai contar o quao popular foi a musica no ano atribuindo pontos a quantas semanas ela ficou entre as mais tocadas
#testando se o char especificado esta na entrada atual
#pode ser usado para identificar anos diferentes

tempo = time.time() #Armazena o tempo de inicio do processamento
topArtistas = {}
topMusicas = {}
for ano in range(2005,2006):
    contador = {}
    topMusicas = {} #ESSA LINHA É POTENCIALMENTE INUTIL #cria um dicionario para guardar o top músicas no geral
    #Salva os registros do ano atual
    for i in range(len(df)):
        
        if (str(ano) + '-') in df['WeekID'][i]: #filtra todas as entradas por ano E mes nesse caso (Maio de 1990)
            if(df['SongID'][i] in contador): #testa se a musica ja esta no dicionario (ja foi computada)
                contador[df['SongID'][i]]['Pontos'] += (101 - df['Week Position'][i])
            else: #cai aqui se ainda nao foi computada
                contador[df['SongID'][i]] = {}
                contador[df['SongID'][i]]['Artista'] = df['Performer'][i]
                contador[df['SongID'][i]]['Titulo'] = df['Song'][i]
                contador[df['SongID'][i]]['Ano'] = ano
                contador[df['SongID'][i]]['Pontos'] = (101 - df['Week Position'][i])
                #A medida 'Peak' nao eh a posicao exata que ela atingiu mas os "pontos maximos" ganhos em uma semana (de 1 a 100)
                contador[df['SongID'][i]]['Peak'] = (101 - df['Peak Position'][i]) #salva qual foi a maior posicao alcançada pela musica (usada para criterios de desempate)
                contador[df['SongID'][i]]['Semanas'] = df['Weeks on Chart'][i] #Terceiro criterio de desempate
        
        
    #Faz o sort do ano atual
    items = sorted(contador.items(), key = lambda tup: (tup[1]["Pontos"], tup[1]["Peak"], tup[1]["Semanas"]), reverse=True)
    
    #armazena as primeiras 200 entradas em um dicionário mais organizado
    top = [dict() for x in range(200)] #cria uma lista de 200 dicionarios
    for i in range(len(top)):
    
        for y in items[i][1]:
            top[i][y] = items[i][1][y]
        
        # Adiciona no dicionário de todos os artistas (ou soma os pontos) para cianção do arquivo topArtistas.bin
        artistaIterado = retiraFeaturing(str(top[i]['Artista']))   # retira o featuring caso tenha
        if artistaIterado in topArtistas:   # testa se o artista já está no dicionário
            topArtistas[artistaIterado] += top[i]['Pontos']   # se sim só soma nos seus pontos
        else:   # caso não esteja, adicona
            topArtistas[artistaIterado] = top[i]['Pontos']     
    
    #Concatena o dicionario no arquivo teste.bin
    filename = 'database.bin'
    topBytes = []
    

    if os.path.exists(filename): #confere se o arquivo existe
        with open(filename,'rb') as openfile:  #with automaticamente da um close() no final
            topBytes = pickle.load(openfile) #carrega o que ja esta salvo no arquivo


    newData = top #Aqui vem a nova data que deve ser concatenada
        
    topBytes = topBytes + newData #concatena o que ja tinha no arquivo binario com a nova data
    with (open(filename, 'wb+')) as openfile: #abre o arquivo no modo de leitura binaria 'wb' como openfile
        pickle.dump(topBytes, openfile) 
        
    #-----------------------------------------------------------------------------    
    #Cria o arquivo de indice para consultas    
    #-----------------------------------------------------------------------------
    filename = 'indices.bin'
    fileBytes = []

    if os.path.exists(filename): #confere se o arquivo existe
        with open(filename,'rb') as openfile:  #with automaticamente da um close() no final
            fileBytes = pickle.load(openfile)


    newData = {'Ano': ano, 'Min': len(topBytes) - 200, 'Max': len(topBytes) - 1} #Aqui vem a nova data que deve ser concatenada
        
    fileBytes.append(newData)#concatena o que ja tinha no arquivo binario com a nova data
    with (open(filename, 'wb+')) as openfile: #abre o arquivo de teste no modo de leitura binaria 'wb' como openfile
        pickle.dump(fileBytes, openfile)
    
#-----------------------------------------------------------------------------
#Ordena o dicionário de artistas
#-----------------------------------------------------------------------------
artistasSorted = sorted(topArtistas.items(), key=lambda kv: kv[1], reverse=True)
    
#Salva ranking geral de artistas
with (open("topArtistas.bin", 'wb+')) as openfile:
    pickle.dump(artistasSorted, openfile)
#-----------------------------------------------------------------------------
#Cria e ordena o topMusicas
#-----------------------------------------------------------------------------
itera = 0
for x in topBytes:
    topMusicas[itera] = x['Pontos']
    itera += 1    
    
topMusicasSorted = sorted(topMusicas.items(), key=lambda kv: kv[1], reverse=True)

#Salva ranking geral de musicas
with (open("topMusicas.bin", 'wb+')) as openfile:
    pickle.dump(topMusicasSorted, openfile)    

#-----------------------------------------------------------------------------
#Cria as arvores TRIE
#-----------------------------------------------------------------------------
raizMusicas = TrieNode('@')   # raiz da trie de músicas 
raizArtistas = TrieNode('@')  # raiz da trie de artistas

indice = 0   # endereço da música no arquivo

for x in topBytes:  
    # o str é necessário para os casos em que a música ou artista são apenas números e interpretados com inteiros
    addString(raizMusicas, str(x['Titulo']), indice)   
    addString(raizArtistas, retiraFeaturing(str(x['Artista'])), indice)
    indice += 1
    
filename = "trieMusicas.bin"
with (open(filename, 'wb+')) as openfile: #abre o arquivo de teste no modo de leitura binaria 'wb' como openfile
    pickle.dump(raizMusicas, openfile)
    
filename = "trieArtistas.bin"
with (open(filename, 'wb+')) as openfile: #abre o arquivo de teste no modo de leitura binaria 'wb' como openfile
    pickle.dump(raizArtistas, openfile)
    
#Retorna o tempo passado desde que o programa comecou a rodar
t = time.time() - tempo #Salva em t o tempo que o processo levou
print('Tempo levado para gerar os arquivos: ', t, 'segundos')

Tempo levado para gerar os arquivos:  3.598292350769043 segundos


### Teste da relação
consultando o indice inicial de um dado ano no arquivo indice e imprimindo os 5 mais populares daquele ano

In [56]:
#Faz a leitura do arquivo de registros
with (open('database.bin', 'rb')) as openfile:
    leitura = pickle.load(openfile)
    
with (open('indices.bin', 'rb')) as openfile:
    indices = pickle.load(openfile)

In [7]:
start = 0

for x in indices:
    if x['Ano'] == 2014:
        start = x['Min']
 
for i in range(5):
    print(leitura[start + i])

{'Artista': 'John Legend', 'Titulo': 'All Of Me', 'Ano': 2014, 'Pontos': 4069, 'Peak': 100, 'Semanas': 59}
{'Artista': 'Pharrell Williams', 'Titulo': 'Happy', 'Ano': 2014, 'Pontos': 3898, 'Peak': 100, 'Semanas': 47}
{'Artista': 'Katy Perry Featuring Juicy J', 'Titulo': 'Dark Horse', 'Ano': 2014, 'Pontos': 3816, 'Peak': 100, 'Semanas': 57}
{'Artista': 'OneRepublic', 'Titulo': 'Counting Stars', 'Ano': 2014, 'Pontos': 3373, 'Peak': 99, 'Semanas': 68}
{'Artista': 'Iggy Azalea Featuring Charli XCX', 'Titulo': 'Fancy', 'Ano': 2014, 'Pontos': 2993, 'Peak': 100, 'Semanas': 39}


In [8]:
start = 0

for x in indices:
    if x['Ano'] == 2014:
        start = x['Min']
        
#Retorna as posicoes de 100 a 104 
desvio = 100

for i in range(5):
    print(leitura[start + i + desvio])

{'Artista': 'Ty Dolla $ign Featuring B.o.B', 'Titulo': 'Paranoid', 'Ano': 2014, 'Pontos': 933, 'Peak': 72, 'Semanas': 20}
{'Artista': 'Fall Out Boy', 'Titulo': 'Centuries', 'Ano': 2014, 'Pontos': 924, 'Peak': 91, 'Semanas': 34}
{'Artista': 'Miley Cyrus', 'Titulo': 'Adore You', 'Ano': 2014, 'Pontos': 920, 'Peak': 80, 'Semanas': 18}
{'Artista': 'Nicki Minaj', 'Titulo': 'Pills N Potions', 'Ano': 2014, 'Pontos': 916, 'Peak': 77, 'Semanas': 18}
{'Artista': 'Miranda Lambert Duet With Carrie Underwood', 'Titulo': "Somethin' Bad", 'Ano': 2014, 'Pontos': 898, 'Peak': 82, 'Semanas': 20}


In [20]:
artistasSorted

[('Madonna', 53607),
 ('Elton John', 47983),
 ('Rihanna', 47334),
 ('Mariah Carey', 46615),
 ('Usher', 43886),
 ('Katy Perry', 37169),
 ('P!nk', 35094),
 ('Michael Jackson', 34889),
 ('Stevie Wonder', 34693),
 ('Taylor Swift', 34497),
 ('The Beatles', 34372),
 ('Whitney Houston', 34084),
 ('Rod Stewart', 33097),
 ('Chris Brown', 32323),
 ('Maroon 5', 32269),
 ('Chicago', 31620),
 ('Beyonce', 30098),
 ('Kelly Clarkson', 30064),
 ('Daryl Hall John Oates', 29523),
 ('The Rolling Stones', 29277),
 ('R. Kelly', 29116),
 ('Billy Joel', 28704),
 ('Janet Jackson', 28575),
 ('The Black Eyed Peas', 28203),
 ('Bee Gees', 28009),
 ('Britney Spears', 26239),
 ('Lady Gaga', 26145),
 ('Drake', 25691),
 ('Aretha Franklin', 25548),
 ('Tim McGraw', 25340),
 ('Eminem', 24376),
 ('Nickelback', 24367),
 ('Kenny Chesney', 23899),
 ('Justin Timberlake', 23766),
 ('Neil Diamond', 23611),
 ('Alicia Keys', 23187),
 ('Pitbull', 23138),
 ('Mary J. Blige', 22902),
 ('Nelly', 22666),
 ('The Beach Boys', 22573),
 ('

In [55]:
topMusicasSorted

[(7400, 4676),
 (10600, 4360),
 (7600, 4208),
 (10400, 4163),
 (7601, 4082),
 (10800, 4069),
 (10200, 3911),
 (8200, 3909),
 (10801, 3898),
 (8800, 3817),
 (10802, 3816),
 (10401, 3725),
 (8000, 3722),
 (7602, 3714),
 (7200, 3662),
 (10000, 3623),
 (9000, 3622),
 (7401, 3614),
 (9400, 3566),
 (8001, 3511),
 (10001, 3485),
 (8801, 3442),
 (7201, 3436),
 (8802, 3424),
 (7603, 3408),
 (10803, 3373),
 (6800, 3372),
 (10201, 3350),
 (8600, 3348),
 (10402, 3338),
 (10202, 3329),
 (10601, 3323),
 (10602, 3318),
 (7604, 3313),
 (7605, 3285),
 (8400, 3282),
 (8201, 3280),
 (8002, 3262),
 (7402, 3259),
 (9001, 3254),
 (10403, 3243),
 (9600, 3207),
 (7202, 3201),
 (7203, 3194),
 (9002, 3143),
 (9800, 3122),
 (7606, 3119),
 (8803, 3114),
 (9801, 3108),
 (8601, 3106),
 (7000, 3099),
 (9401, 3090),
 (9402, 3065),
 (10603, 3060),
 (9200, 3059),
 (9201, 3055),
 (8003, 3033),
 (9802, 3028),
 (7800, 3024),
 (8004, 3023),
 (10604, 3003),
 (8401, 3002),
 (7607, 2998),
 (10804, 2993),
 (10605, 2988),
 (900

In [60]:
print(len(artistasSorted))
print(len(topMusicasSorted))
print(len(leitura))

3701
11000
11000


In [58]:
print(leitura[7400])
print(leitura[10600])
print(leitura[7600])
print(leitura[10400])
print(leitura[7601])
print(leitura[10800])
print(leitura[10200])

{'Artista': 'Jewel', 'Titulo': 'Foolish Games/You Were Meant For Me', 'Ano': 1997, 'Pontos': 4676, 'Peak': 99, 'Semanas': 65}
{'Artista': 'Imagine Dragons', 'Titulo': 'Radioactive', 'Ano': 2013, 'Pontos': 4360, 'Peak': 98, 'Semanas': 87}
{'Artista': 'Savage Garden', 'Titulo': 'Truly Madly Deeply', 'Ano': 1998, 'Pontos': 4208, 'Peak': 100, 'Semanas': 52}
{'Artista': 'Gotye Featuring Kimbra', 'Titulo': 'Somebody That I Used To Know', 'Ano': 2012, 'Pontos': 4163, 'Peak': 100, 'Semanas': 59}
{'Artista': 'Next', 'Titulo': 'Too Close', 'Ano': 1998, 'Pontos': 4082, 'Peak': 100, 'Semanas': 53}
{'Artista': 'John Legend', 'Titulo': 'All Of Me', 'Ano': 2014, 'Pontos': 4069, 'Peak': 100, 'Semanas': 59}
{'Artista': 'Adele', 'Titulo': 'Rolling In The Deep', 'Ano': 2011, 'Pontos': 3911, 'Peak': 100, 'Semanas': 65}


In [12]:
print(type(df))
print(df.columns)
print("\n", df['Song'][0])

<class 'pandas.core.frame.DataFrame'>
Index(['url', 'WeekID', 'Week Position', 'Song', 'Performer', 'SongID',
       'Instance', 'Previous Week Position', 'Peak Position',
       'Weeks on Chart'],
      dtype='object')

 "B" Girls


<class 'pandas.core.frame.DataFrame'>
Index(['url', 'Song'], dtype='object')

 BBBBB GURLS
